## Setting up

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
from tqdm import tqdm
from selenium import webdriver
import chromedriver_binary

C:\Users\Abdullah Fazili\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
path = chromedriver_binary.chromedriver_filename

In [4]:
inp = input("Job: ").lower()

Job: Data Scientist


## Temporary web driver to extract number of pages

In [5]:
link = "https://www.naukri.com/"+inp.replace(" ","-") +"-jobs-"
temp = webdriver.Chrome(path)
temp.get(link)
soup = BeautifulSoup(temp.page_source,'html.parser')


<ipython-input-5-660c7b75ed88>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  temp = webdriver.Chrome(path)


<bound method Tag.prettify of <html lang="en"><head><title>Data Scientist Jobs, 765 Data Scientist Job Vacancies In August 2022 - Naukri.com</title><meta content="Apply To 765 Data Scientist Jobs On Naukri.com, #1 Job Portal In India. Explore Data Scientist Job Openings In Your Desired Locations Now!" name="description"/><link href="android-app://naukriApp.appModules.login/https/www.naukri.com/data-scientist-jobs" rel="alternate"/><link href="https://www.naukri.com/data-scientist-jobs" rel="canonical"/><meta content="jobsearch" name="atdlayout"/><meta content="ALL" name="robots"/><meta content="Jobs &amp; Career: Job Search, Apply Jobs, Post Jobs" name="classification"/><meta content="no-cache" name="pragma"/><meta content="general" name="rating"/><meta content="1 day" name="revisit-after"/><meta content="GLOBAL" name="distribution"/><meta content="document" name="resource-type"/><meta content="www.naukri.com" name="author"/><meta content="EN" name="content-language"/><meta content="17

## Number of web pages for a job

In [6]:
x= soup.find('span',class_='fleft grey-text mr-5 fs12').text[10:15]
x = x.strip(' ')
x = round(int(x)/20)
x

38

## Generating a list of all URL's for a given job

- A list conatining link of all pages for a given job

In [ ]:
base_url  = "https://www.naukri.com/"

all_urls = list()

def generate_urls():
    for i in range(0,x+1):
        all_urls.append(base_url + inp.replace(" ","-") +"-jobs-"+ str(i))

In [102]:
generate_urls()
all_urls[1:10]

['https://www.naukri.com/data-scientist-jobs-1',
 'https://www.naukri.com/data-scientist-jobs-2',
 'https://www.naukri.com/data-scientist-jobs-3',
 'https://www.naukri.com/data-scientist-jobs-4',
 'https://www.naukri.com/data-scientist-jobs-5',
 'https://www.naukri.com/data-scientist-jobs-6',
 'https://www.naukri.com/data-scientist-jobs-7',
 'https://www.naukri.com/data-scientist-jobs-8',
 'https://www.naukri.com/data-scientist-jobs-9']

## A function used to open multiple windows to scrap data from each window

 * This function has a list **lis** and another list **lis2**
 * **lis** is used to open multiple chrome windows as lis saves them as browser1,browser2 and so on
 * **lis2** is used to create a BeautifulSoup for each individual window it is necessary as using same BeautifulSoup leads    to repetition of data
 * Finally the data is saved and returned as well

In [93]:
def multiple_browser():
    lis  = []
    lis2 = []
    Data = []
    for i in tqdm(range(x+1)):

        browser = (('browser'+str(i)))
        lis.append([browser])
        lis[i]  = webdriver.Chrome(path)
        lis[i].get(str(all_urls[i]))
        soup = (('soup'+str(i)))
        lis2.append([soup])
        lis2[i]  = BeautifulSoup(lis[i].page_source,'html.parser')
        time.sleep(2)
        for sp in lis2[i].find_all('article',class_='jobTuple bgWhite br4 mb-8'):
            try:
                title               = sp.find('a',class_='title fw500 ellipsis').text
            except:
                title               = np.nan
            try:
                company_name        = (sp.find('a',class_='subTitle ellipsis fleft').get('title'))
            except:
                company_name        = np.nan    
            try:    
                link                = sp.find('a',class_='title fw500 ellipsis').get('href')
            except:
                link                = np.nan
            try:    
                rating              = soup.find('span',class_='starRating fleft dot').text
            except:
                rating              = np.nan
            try:
                Review_page_company = sp.find('a',class_='reviewsCount ml-5 fleft blue-text').get('href')
            except:
                Review_page_company = np.nan
            try:    
                Rating_Company      = sp.find('a',class_='reviewsCount ml-5 fleft blue-text').text
            except:
                Rating_Company      = np.nan
            try:
                experience          = sp.find('span',class_='ellipsis fleft fs12 lh16').get('title')
            except:
                experience          = np.nan
            try:    
                salary              = sp.find('li',class_='fleft grey-text br2 placeHolderLi salary').text
            except:
                salary              = np.nan
            try:
                Location            = sp.find('li',class_ = 'fleft grey-text br2 placeHolderLi location').text
            except:
                Location            = np.nan
            try:
                Skill_Set           = sp.find('ul',class_ = 'tags has-description').text
            except:
                Skill_Set           = np.nan

            Data.append([title,company_name,rating,experience,salary,Rating_Company,Location,Skill_Set,link,Review_page_company])
    df = pd.DataFrame(Data,columns=['Title','Company','Rating','Experience','Salary','Rating of Company','Location','Skill_Set','Link','Review_page_company'])
    df.to_csv("Multi_Windows_Scrap.csv",index=False)
    return df

In [94]:
multiple_browser()

<ipython-input-93-0f452ca11e66>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  lis[i]  = webdriver.Chrome(path)


,Title,Company,Rating,Experience,Salary,Rating of Company,Location,Skill_Set,Link,Review_page_company
0,Analystics & Modeling Specialist,Accenture,NaN,6-8 Yrs,Not disclosed,(30728 Reviews),"Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...",BfsiConsultingMachine learningOpen sourcePytho...,https://www.naukri.com/job-listings-analystics...,https://www.ambitionbox.com/reviews/accenture-...
1,Job||Job Opening For AI Technologist - Data Sc...,Wipro,NaN,5-10 Yrs,Not disclosed,(26719 Reviews),"New Delhi, Hyderabad/Secunderabad, Pune, Chenn...",deep learningpythonAlgorithmsNatural Language ...,https://www.naukri.com/job-listings-opening-fo...,https://www.ambitionbox.com/reviews/wipro-revi...
2,Hiring For DATA Scientist @ NTT DATA Business ...,NTT DATA Business Solutions Private Limited,NaN,4-9 Yrs,"18,00,000 - 20,00,000 PA.",(57 Reviews),"Noida, Hyderabad/Secunderabad, Pune, Gurgaon/G...",Data ScienceArtificial IntelligenceMachine Lea...,https://www.naukri.com/job-listings-hiring-for...,https://www.ambitionbox.com/reviews/ntt-data-b...
3,Lead ML Scientist,Fractal Analytics,NaN,6-10 Yrs,Not disclosed,(291 Reviews),"Mumbai, Bangalore/Bengaluru",deep learningLinuxdata scienceAnalyticalMachin...,https://www.naukri.com/job-listings-lead-ml-sc...,https://www.ambitionbox.com/reviews/fractal-an...
4,Senior Manager - EmTech - Machine Learning - P&T,PwC,NaN,5-12 Yrs,Not disclosed,(3837 Reviews),"Mumbai, Hyderabad/Secunderabad, Bangalore/Beng...",AutomationPrototypeSimulationAnalyticalProcess...,https://www.naukri.com/job-listings-senior-man...,https://www.ambitionbox.com/reviews/pwc-review...
...,...,...,...,...,...,...,...,...,...,...
515,DATA Scientist - New Delhi - Leading MNC,Advance Talent Services,NaN,1-6 Yrs,Not disclosed,NaN,Delhi / NCR,DATA SCIENTISTPythonTableauHadoopQlikViewData ...,https://www.naukri.com/job-listings-data-scien...,NaN
516,"MNC Hiring For "" Sr. Data Scientist + cloud "" ...",ESS CEE HR SOLUTIONS,NaN,3-7 Yrs,"8,00,000 - 14,00,000 PA.",NaN,Gurgaon/Gurugram,model deploymentmachine learningAzure/ AWSNLPC...,https://www.naukri.com/job-listings-mnc-hiring...,NaN
517,Data Scientist,SHERVANI TECH HUB PRIVATE LIMITED,NaN,"12,00,000 - 22,00,000 PA.","12,00,000 - 22,00,000 PA.",NaN,Remote,data modelsdata architecturesData ScientistPyt...,https://www.naukri.com/job-listings-data-scien...,NaN
518,Data Scientist,Sydata Consulting India Pvt Ltd,NaN,Not disclosed,Not disclosed,NaN,"Bangalore/Bengaluru, Delhi / NCR, Mumbai (All ...",pythonSparkAWSIT SkillsData ScienceMachine Lea...,https://www.naukri.com/job-listings-data-scien...,NaN


In [122]:
df_1 = pd.read_csv('Multi_Windows_Scrap.csv')

In [123]:
df_1['Link']

0      https://www.naukri.com/job-listings-analystics...
1      https://www.naukri.com/job-listings-opening-fo...
2      https://www.naukri.com/job-listings-hiring-for...
3      https://www.naukri.com/job-listings-lead-ml-sc...
4      https://www.naukri.com/job-listings-senior-man...
                             ...                        
515    https://www.naukri.com/job-listings-data-scien...
516    https://www.naukri.com/job-listings-mnc-hiring...
517    https://www.naukri.com/job-listings-data-scien...
518    https://www.naukri.com/job-listings-data-scien...
519    https://www.naukri.com/job-listings-data-scien...
Name: Link, Length: 520, dtype: object

### Scraping from individual posts using multiple windows

In [151]:
length = len(df_1['Link'])
def depth_post_scrape():
    lis  = []
    data = []
    for i in tqdm(range(length)):
        browser = (('browser'+str(i)))
        lis.append([browser])
        lis[i]  = webdriver.Chrome(path)
        lis[i].get(str(df1['Link'][i]))

        soup  = BeautifulSoup(lis[i].page_source,'html.parser')
        
        try:
            Average_Salary = soup.find('span',class_='avg-salary-inner').text
        except:
            Average_Salary = np.nan
        try:    
            Role           = soup.find('div',class_='other-details').text.split(',')
        except:
            Role           = np.nan
        try:    
            Education      = soup.find('div',class_='education').text[9:]
        except:
            Education      = np.nan
        data.append([Average_Salary,Role,Education])    
    df = pd.DataFrame(Data,columns=['Expected Average Salary','Role','Education'])
    df.to_csv("Multi_Jobs_Depth_Scrap.csv",index=False)
    return df
    
        
        

In [ ]:
depth_post_scrape()

### Traditional Way

In [157]:
browser = webdriver.Chrome(path)

<ipython-input-157-8996e59d0d1c>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(path)


In [158]:
data = []
for i in tqdm(df_1['Link']):
    browser.get(i)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    time.sleep(0.5)
    try:
        Average_Salary = soup.find('span',class_='avg-salary-inner').text
    except:
        Average_Salary = np.nan
    try:    
        Role           = soup.find('div',class_='other-details').text.split(',')
    except:
        Role           = np.nan
    try:    
        Education      = soup.find('div',class_='education').text[9:]
    except:
        Education      = np.nan
    data.append([Average_Salary,Role,Education])    
    

100%|████████████████████████████████████████████████████████████████████████████████| 520/520 [12:25<00:00,  1.43s/it]


In [160]:
df = pd.DataFrame(data,columns=['Expected Average Salary','Role','Education'])
df.to_csv("Multi_Jobs_Depth_Scrap.csv",index=False)

### Creating a common Dataframe

In [161]:
df_common = pd.DataFrame()

In [162]:
df_common['Company']                  = df_1['Company']
df_common['Title']                    = df_1['Title']
df_common['Role']                     = df['Role']
df_common['Experience']               = df_1['Experience']
df_common['Real Salary']              = df_1['Salary']
df_common['Expected Salary']          = df['Expected Average Salary']
df_common['Education']                = df['Education']
df_common['Company Rating']           = df_1['Rating']
df_common['Reviews of Company']       = df_1['Rating of Company']
df_common['Location']                 = df_1['Location']
df_common['Skills']                   = df_1['Skill_Set']
df_common['Link']                     = df_1['Link']
df_common['Review Page of Company']   = df_1['Review_page_company']

In [163]:
df_1.to_csv("Data_Scientist_Jobs_main_final.csv",index=False)

In [164]:
df_1

,Title,Company,Rating,Experience,Salary,Rating of Company,Location,Skill_Set,Link,Review_page_company
0,Analystics & Modeling Specialist,Accenture,NaN,6-8 Yrs,Not disclosed,(30728 Reviews),"Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...",BfsiConsultingMachine learningOpen sourcePytho...,https://www.naukri.com/job-listings-analystics...,https://www.ambitionbox.com/reviews/accenture-...
1,Job||Job Opening For AI Technologist - Data Sc...,Wipro,NaN,5-10 Yrs,Not disclosed,(26719 Reviews),"New Delhi, Hyderabad/Secunderabad, Pune, Chenn...",deep learningpythonAlgorithmsNatural Language ...,https://www.naukri.com/job-listings-opening-fo...,https://www.ambitionbox.com/reviews/wipro-revi...
2,Hiring For DATA Scientist @ NTT DATA Business ...,NTT DATA Business Solutions Private Limited,NaN,4-9 Yrs,"18,00,000 - 20,00,000 PA.",(57 Reviews),"Noida, Hyderabad/Secunderabad, Pune, Gurgaon/G...",Data ScienceArtificial IntelligenceMachine Lea...,https://www.naukri.com/job-listings-hiring-for...,https://www.ambitionbox.com/reviews/ntt-data-b...
3,Lead ML Scientist,Fractal Analytics,NaN,6-10 Yrs,Not disclosed,(291 Reviews),"Mumbai, Bangalore/Bengaluru",deep learningLinuxdata scienceAnalyticalMachin...,https://www.naukri.com/job-listings-lead-ml-sc...,https://www.ambitionbox.com/reviews/fractal-an...
4,Senior Manager - EmTech - Machine Learning - P&T,PwC,NaN,5-12 Yrs,Not disclosed,(3837 Reviews),"Mumbai, Hyderabad/Secunderabad, Bangalore/Beng...",AutomationPrototypeSimulationAnalyticalProcess...,https://www.naukri.com/job-listings-senior-man...,https://www.ambitionbox.com/reviews/pwc-review...
...,...,...,...,...,...,...,...,...,...,...
515,DATA Scientist - New Delhi - Leading MNC,Advance Talent Services,NaN,1-6 Yrs,Not disclosed,NaN,Delhi / NCR,DATA SCIENTISTPythonTableauHadoopQlikViewData ...,https://www.naukri.com/job-listings-data-scien...,NaN
516,"MNC Hiring For "" Sr. Data Scientist + cloud "" ...",ESS CEE HR SOLUTIONS,NaN,3-7 Yrs,"8,00,000 - 14,00,000 PA.",NaN,Gurgaon/Gurugram,model deploymentmachine learningAzure/ AWSNLPC...,https://www.naukri.com/job-listings-mnc-hiring...,NaN
517,Data Scientist,SHERVANI TECH HUB PRIVATE LIMITED,NaN,"12,00,000 - 22,00,000 PA.","12,00,000 - 22,00,000 PA.",NaN,Remote,data modelsdata architecturesData ScientistPyt...,https://www.naukri.com/job-listings-data-scien...,NaN
518,Data Scientist,Sydata Consulting India Pvt Ltd,NaN,Not disclosed,Not disclosed,NaN,"Bangalore/Bengaluru, Delhi / NCR, Mumbai (All ...",pythonSparkAWSIT SkillsData ScienceMachine Lea...,https://www.naukri.com/job-listings-data-scien...,NaN
